In [ ]:
#| default_exp serializers

# Serializers
> Various utilities to encode MARIS dataset as `NetCDF`, `csv`, ... formats.

In [ ]:
#| export
from netCDF4 import Dataset
import pandas as pd
from typing import Dict

In [ ]:
#| export
def to_netcdf(
    df:pd.DataFrame, # Dataframe to encode
    fname_cdl:str, # File name and path to the MARIS CDL template
    fname_output:str, # Name of output file to produce
    cfgs:Dict, # Config file containing global attributes
):
    "Encode MARIS dataset (provided as Pandas DataFrame) to NetCDF file"
    with Dataset(fname_cdl) as src, Dataset(fname_output, 'w', format='NETCDF3_CLASSIC') as dst:
        # copy global attributes all at once via dictionary
        dst.setncatts(src.__dict__)

        dst.setncatts(cfgs['global_attr']) 
        # copy dimensions
        for name, dimension in src.dimensions.items():
            dst.createDimension(
                name, (len(dimension) if not dimension.isunlimited() else None))

        n_before = 0
        n_after = 0
        # copy all variables of interest and fill them
        for name_var_src, var_src in src.variables.items():
            if name_var_src in df.reset_index().columns:
                x = dst.createVariable(name_var_src, var_src.datatype, var_src.dimensions,
                                       compression='zlib', complevel=9)
                # fill variables
                # Sanitize
                n_before += sum(df.reset_index()[name_var_src].notna())
                df_sanitized = pd.to_numeric(df.reset_index()[name_var_src], 
                                             errors='coerce', downcast=None)
                n_after += sum(df_sanitized.notna())
                dst[name_var_src][:] = df_sanitized .values
                # copy variable attributes all at once via dictionary
                dst[name_var_src].setncatts(src[name_var_src].__dict__)
                dst[name_var_src].unit = cfgs['global_attr']['unit']
        print(f'% of discarded data: {100*(n_before - n_after)/n_before}')